In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import gensim
import numpy as np
import warnings
from gensim import corpora, models
from sklearn.metrics import confusion_matrix, accuracy_score
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
posts_df = pd.read_pickle('../data/interum/text_target.pkl')

In [3]:
# convert into features and target 
feature = posts_df['tokens']
label = posts_df['target']  # label 

In [4]:
# more words to remove and remove words longer than length 2
words = ['class', 'file', 'use', 'code',
         'string', 'function', 'new', 'like', 
         'way', 'would', 'name', 'get', 'list',
         'want', 'value','return','data']
feature = feature.apply(
    lambda x: [w for w in x if len(w) > 2 and w not in words])

## feature engineering for clustering

In [5]:
# dictionary for train 
dictionary = gensim.corpora.Dictionary(feature)

In [6]:
# create bag of words 
bow = [dictionary.doc2bow(doc) for doc in feature]


In [7]:
# tfidf for bow 
tfidf = models.TfidfModel(bow)


In [8]:
corpus_tfidf = tfidf[bow]

In [9]:
# generate a model for bag of words 
# we know we have 5 tops 
lda_model = gensim.models.LdaMulticore(
    bow, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [10]:
lda_model.print_topics() # notice many filer words 

[(0,
  '0.010*"version" + 0.009*"error" + 0.009*"run" + 0.009*"project" + 0.008*"lib" + 0.007*"build" + 0.007*"library" + 0.006*"work" + 0.006*"try" + 0.006*"self"'),
 (1,
  '0.012*"http" + 0.009*"user" + 0.008*"com" + 0.008*"page" + 0.007*"org" + 0.007*"work" + 0.007*"try" + 0.006*"script" + 0.006*"application" + 0.006*"request"'),
 (2,
  '0.012*"system" + 0.011*"test" + 0.011*"int" + 0.008*"size" + 0.007*"add" + 0.006*"type" + 0.006*"time" + 0.005*"run" + 0.005*"array" + 0.005*"object"'),
 (3,
  '0.013*"int" + 0.009*"public" + 0.009*"method" + 0.006*"one" + 0.006*"time" + 0.005*"need" + 0.005*"question" + 0.005*"thread" + 0.005*"know" + 0.005*"void"'),
 (4,
  '0.013*"public" + 0.013*"object" + 0.010*"var" + 0.010*"type" + 0.008*"div" + 0.008*"foo" + 0.007*"std" + 0.007*"null" + 0.007*"try" + 0.006*"method"')]

## tfidf LDA model

In [11]:
lda_model_tfidf = gensim.models.LdaMulticore(
    corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)

In [12]:
lda_model_tfidf.print_topics()

[(0,
  '0.002*"page" + 0.002*"jquery" + 0.002*"script" + 0.002*"user" + 0.002*"application" + 0.002*"html" + 0.002*"image" + 0.002*"http" + 0.002*"work" + 0.002*"need"'),
 (1,
  '0.003*"array" + 0.003*"int" + 0.002*"element" + 0.002*"text" + 0.002*"div" + 0.002*"type" + 0.002*"convert" + 0.002*"var" + 0.002*"byte" + 0.002*"object"'),
 (2,
  '0.003*"foo" + 0.003*"project" + 0.002*"int" + 0.002*"eclipse" + 0.002*"error" + 0.002*"run" + 0.002*"test" + 0.002*"jar" + 0.002*"version" + 0.002*"maven"'),
 (3,
  '0.001*"std" + 0.001*"object" + 0.001*"int" + 0.001*"char" + 0.001*"text" + 0.001*"var" + 0.001*"call" + 0.001*"true" + 0.001*"method" + 0.001*"example"'),
 (4,
  '0.003*"object" + 0.003*"method" + 0.003*"int" + 0.003*"date" + 0.003*"public" + 0.003*"type" + 0.002*"time" + 0.002*"test" + 0.002*"static" + 0.002*"one"')]

In [13]:
pred_bow = []
for i in bow:
    result = lda_model[i]
    if len(result) > 1:
        pred_bow.append(sorted(result, key=lambda x: x[1], reverse = True )[0][0])
    else:
        pred_bow.append(result[0][0])    

In [14]:
pred_tdif = []
for i in bow:
    result = lda_model_tfidf[i]
    if len(result) > 1:
        pred_tdif.append(sorted(result, key=lambda x: x[1], reverse = True )[0][0])
    else:
        pred_tdif.append(result[0][0])
        

In [15]:
result = pd.DataFrame([pred_bow,pred_tdif,label]).T

In [16]:
result.columns = ['bow','tdif','true_label']

In [17]:
result.true_label.unique

<bound method Series.unique of 0        javascript
1               c++
2        javascript
3            python
4        javascript
            ...    
30297            c#
30298          java
30299        python
30300          java
30301    javascript
Name: true_label, Length: 30302, dtype: object>

In [18]:
mask1 = result.true_label =='c++'
mask2 = result.true_label == 'javascript'
mask3 = result.true_label =='c#'
mask4= result.true_label == 'python'
mask5 = result.true_label =='java'

## bow:

* c++: topic 3
* javascript: topic 1
* c#: topic 4
* python:topic 2
* java: topic 0

In [19]:
result[mask1]['bow'].value_counts(normalize = True)

3    0.585762
4    0.162755
2    0.146247
0    0.091824
1    0.013412
Name: bow, dtype: float64

In [20]:
result[mask2]['bow'].value_counts(normalize = True)

1    0.387529
4    0.340870
3    0.172667
0    0.054868
2    0.044067
Name: bow, dtype: float64

In [21]:
result[mask3]['bow'].value_counts(normalize = True)

3    0.340968
4    0.259376
1    0.172774
2    0.139708
0    0.087174
Name: bow, dtype: float64

In [22]:
result[mask4]['bow'].value_counts(normalize = True)

3    0.343682
0    0.231642
1    0.166965
4    0.149055
2    0.108657
Name: bow, dtype: float64

In [23]:
result[mask5]['bow'].value_counts(normalize = True)

3    0.402410
0    0.211780
4    0.177510
1    0.144043
2    0.064257
Name: bow, dtype: float64

## tdif:

* c++: topic 4
* javascript: topic 0 
* c#: topic 2
* python:topic 1
* java: topic 3

In [24]:
result[mask1]['tdif'].value_counts(normalize = True)

4    0.702347
0    0.137477
2    0.116069
1    0.039721
3    0.004385
Name: tdif, dtype: float64

In [25]:
result[mask2]['tdif'].value_counts(normalize = True)

0    0.605703
4    0.265985
1    0.105415
2    0.019729
3    0.003168
Name: tdif, dtype: float64

In [26]:
result[mask3]['tdif'].value_counts(normalize = True)

4    0.555397
0    0.314200
2    0.070713
1    0.053822
3    0.005869
Name: tdif, dtype: float64

In [27]:
result[mask4]['tdif'].value_counts(normalize = True)

4    0.540896
0    0.319403
2    0.096318
1    0.040000
3    0.003383
Name: tdif, dtype: float64

In [28]:
result[mask5]['tdif'].value_counts(normalize = True)

4    0.532262
0    0.277108
2    0.137483
1    0.050067
3    0.003079
Name: tdif, dtype: float64

c++: topic 4
* javascript: topic 0 
* c#: topic 2
* python:topic 1
* java: topic 3

In [30]:
bow_map = {0:'java',3:'c++',1:'javascript',4:'c#',2:'python'}
tdif_map = {0:'javascript',4:'c++',2:'c#',1:'python',3:'java'}

result.bow = result.bow.map(bow_map)
result.tdif = result.tdif.map(tdif_map)

In [31]:
result

,bow,tdif,true_label
0,c++,c++,javascript
1,c++,c#,c++
2,javascript,javascript,javascript
3,c++,c++,python
4,c++,c++,javascript
5,c++,c++,javascript
6,c#,javascript,javascript
7,c++,javascript,javascript
8,c++,c++,java
9,c++,c++,c++


# confusion matrix bow

In [32]:
confusion_matrix(result.true_label,result.bow)

array([[1812, 2382,  609, 1207,  976],
       [ 631, 2271,  356,   52,  567],
       [1326, 3006, 1582, 1076,  480],
       [2367, 1199,  381, 2691,  306],
       [ 749, 1727, 1164,  839,  546]])

In [33]:
accuracy_score(result.true_label,result.bow)

0.293775988383605

In [34]:
confusion_matrix(result.true_label, result.tdif)

array([[ 494, 3880,   41, 2195,  376],
       [ 450, 2723,   17,  533,  154],
       [1027, 3976,   23, 2070,  374],
       [ 137, 1847,   22, 4206,  732],
       [ 484, 2718,   17, 1605,  201]])

In [35]:
accuracy_score(result.true_label,result.tdif)

0.2523595802257277